# VacationPy

In [53]:
 # Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys1 import g_key



## Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [54]:
file = 'City_Weather_Data.csv'
city_df = pd.read_csv(file)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Caibarién,22.5206,-79.4669,84.67,69,92,4.38,CU,1635012157
1,Draguignan,43.5333,6.4667,62.80,79,1,0.00,FR,1635012157
2,Mitchell,43.7094,-98.0298,50.11,60,100,15.99,US,1635012037
3,Khatanga,71.9667,102.5000,10.92,92,100,17.07,RU,1635012158
4,Laguna,38.4210,-121.4238,64.11,77,75,4.00,US,1635012158


## Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [55]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


# Ensure LATs/LNGs/Humidity are in float type and dropna

locations = city_df[["Lat", "Lng"]].astype(float)
#city_df = city_df.dropna()
humidity_perc = city_df['Humidity'].astype(float)

# Create a Humidity Heatmap Layer

fig =  gmaps.figure()

humid_layer = gmaps.heatmap_layer(locations, 
                                  weights=humidity_perc, 
                                  dissipating=False, 
                                  max_intensity=100,
                                  point_radius = 1)

fig.add_layer(humid_layer)
fig



Figure(layout=FigureLayout(height='420px'))

##  Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [96]:
hotel_df = city_df.loc[(city_df['Max Temp']<80)&
                       (city_df['Max Temp']>=70)&
                       (city_df['Wind Speed']<10)&
                       (city_df['Cloudiness']==0)
                      ]
hotel_df.head(25)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
163,Maputo,-25.9653,32.5892,76.68,83,0,5.01,MZ,1635012206
167,Sainte-Suzanne,-20.9061,55.6089,75.42,84,0,9.22,RE,1635012207
280,Avera,33.1940,-82.5271,74.68,41,0,4.88,US,1635012244
386,Daulatpur,26.5015,67.9706,78.62,12,0,4.47,PK,1635012276
434,Bannu,32.9854,70.6027,71.40,40,0,1.59,PK,1635012289
459,Kiryat Gat,31.6100,34.7642,72.81,100,0,6.85,IL,1635012297
537,Saint-Pierre,-21.3393,55.4781,74.88,84,0,2.30,RE,1635012004
548,Gobabis,-22.4500,18.9667,72.99,7,0,7.18,NaN,1635012326
570,Marsá Maţrūḩ,31.3525,27.2453,70.30,67,0,7.72,EG,1635012334


##  Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [99]:
# Add Empty Column "Hotel Name"

hotel_df['Hotel Name'] = ''
hotel_df.head(30)

<ipython-input-99-b74a2e23d9f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
163,Maputo,-25.9653,32.5892,76.68,83,0,5.01,MZ,1635012206,
167,Sainte-Suzanne,-20.9061,55.6089,75.42,84,0,9.22,RE,1635012207,
280,Avera,33.1940,-82.5271,74.68,41,0,4.88,US,1635012244,
386,Daulatpur,26.5015,67.9706,78.62,12,0,4.47,PK,1635012276,
434,Bannu,32.9854,70.6027,71.40,40,0,1.59,PK,1635012289,
459,Kiryat Gat,31.6100,34.7642,72.81,100,0,6.85,IL,1635012297,
537,Saint-Pierre,-21.3393,55.4781,74.88,84,0,2.30,RE,1635012004,
548,Gobabis,-22.4500,18.9667,72.99,7,0,7.18,NaN,1635012326,
570,Marsá Maţrūḩ,31.3525,27.2453,70.30,67,0,7.72,EG,1635012334,


In [100]:
# For Loop To call API Hotel from Google

for cnt in range(len(hotel_df['City'])):
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "location": str(hotel_df.iloc[cnt,1])+','+str(hotel_df.iloc[cnt,2]), 
            "radius": 5000,
            "type": "lodging",
            "key": g_key,
            }

        response = requests.get(base_url,params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_df.iloc[cnt,9] = hotel_name
    except:
        hotel_df.iloc[cnt,9] = ''
        
hotel_df.head(20)


C:\Users\drewp\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
163,Maputo,-25.9653,32.5892,76.68,83,0,5.01,MZ,1635012206,Pestana Rovuma
167,Sainte-Suzanne,-20.9061,55.6089,75.42,84,0,9.22,RE,1635012207,Gites Ango
280,Avera,33.1940,-82.5271,74.68,41,0,4.88,US,1635012244,
386,Daulatpur,26.5015,67.9706,78.62,12,0,4.47,PK,1635012276,SHAHJEE HOUSE
434,Bannu,32.9854,70.6027,71.40,40,0,1.59,PK,1635012289,AL JANNAT STUDENTS HOSTEL BANNU
459,Kiryat Gat,31.6100,34.7642,72.81,100,0,6.85,IL,1635012297,OlusHome
537,Saint-Pierre,-21.3393,55.4781,74.88,84,0,2.30,RE,1635012004,Lindsey Hôtel
548,Gobabis,-22.4500,18.9667,72.99,7,0,7.18,NaN,1635012326,Goba Lodge
570,Marsá Maţrūḩ,31.3525,27.2453,70.30,67,0,7.72,EG,1635012334,Beau Site


In [102]:
# Drop Missing Hotel Row by Index

hotel_df = hotel_df.drop(hotel_df.index[[2]])
hotel_df.head(25)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
163,Maputo,-25.9653,32.5892,76.68,83,0,5.01,MZ,1635012206,Pestana Rovuma
167,Sainte-Suzanne,-20.9061,55.6089,75.42,84,0,9.22,RE,1635012207,Gites Ango
434,Bannu,32.9854,70.6027,71.40,40,0,1.59,PK,1635012289,AL JANNAT STUDENTS HOSTEL BANNU
459,Kiryat Gat,31.6100,34.7642,72.81,100,0,6.85,IL,1635012297,OlusHome
537,Saint-Pierre,-21.3393,55.4781,74.88,84,0,2.30,RE,1635012004,Lindsey Hôtel
548,Gobabis,-22.4500,18.9667,72.99,7,0,7.18,NaN,1635012326,Goba Lodge
570,Marsá Maţrūḩ,31.3525,27.2453,70.30,67,0,7.72,EG,1635012334,Beau Site


In [118]:
# Using the template add the hotel marks to the heatmap

info_box_template = """
Name
{Hotel Name}
City
{City}
Country
{Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations,hotel_info)

# Add Locations to Heatmap

fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))